In [ ]:
using DiffEqBase
using Sundials
using DiffEqSensitivity
using PythonPlot
using ReactionMechanismSimulator

In [ ]:
phaseDict = readinput("../src/testing/superminimal.rms") #load mechanism dictionary

In [ ]:
spcs = phaseDict["phase"]["Species"]; #mechanism dictionaries index:  phaseDict[phasename]["Species" or "Reactions"]
rxns = phaseDict["phase"]["Reactions"];

In [ ]:
ig = IdealGas(spcs,rxns,name="gas"); #Define the phase (how species thermodynamic and kinetic properties calculated)
initialconds = Dict(["T"=>1000.0,"P"=>1e5,"H2"=>0.67,"O2"=>0.33]); #Set simulation Initial Temp and Pressure
domain,y0,p = ConstantTPDomain(phase=ig,initialconds=initialconds); #Define the domain (encodes how system thermodynamic properties calculated)

In [ ]:
react = Reactor(domain,y0,(0.0,150.2);p=p); #Create the reactor object

In [ ]:
sol = solve(react.ode,CVODE_BDF(),abstol=1e-20,reltol=1e-12); #solve the ode associated with the reactor

In [ ]:
bsol = Simulation(sol,domain);

In [ ]:
plotmolefractions(bsol,150.0; t0=1e-15, N=1000, tol=0.01, exclude=["O2"])

In [ ]:
plotrops(bsol,"OH",tol=0.01)

In [ ]:
#Adjoint Sensitivities
dps = getadjointsensitivities(bsol,"H2",CVODE_BDF();sensealg=InterpolatingAdjoint(autojacvec=ReverseDiffVJP(true)),abstol=1e-8,reltol=1e-6)

In [ ]:
plotthermoadjointsensitivities(bsol,"H2",dps)

In [ ]:
plotrateadjointsensitivities(bsol,"H2",dps)

In [ ]:
#Forward Sensitivities
react2 = Reactor(domain,y0,(0.0,150.2);p=p,forwardsensitivities=true);
sol2 = solve(react2.ode,CVODE_BDF(),abstol=1e-20,reltol=1e-6);
bsol2 = Simulation(sol2,domain);

In [ ]:
plotmaxthermoforwardsensitivity(bsol2,"H2")

In [ ]:
plotmaxrateforwardsensitivity(bsol2,"H2")

In [ ]:
#Threaded Sensitivities
react3 = Reactor(domain,y0,(0.0,150.2);p=p); 
sol3 = threadedsensitivities(react3;odekwargs=Dict([:abstol=>1e-21,:reltol=>1e-6]),
        senskwargs=Dict([:abstol=>1e-21,:reltol=>1e-6]));
bsol3 = Simulation(sol3,domain);

In [ ]:
plotmaxthermoforwardsensitivity(bsol3,"H2")

In [ ]:
plotmaxrateforwardsensitivity(bsol3,"H2")